In [37]:
import pandas as pd
import numpy as np

In [38]:
movies_df = pd.read_csv('data/movies_metadata.csv',usecols=['id','title'])

rating_df = pd.read_csv('data/ratings_small.csv',usecols=['userId', 'movieId', 'rating'],
dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})

In [39]:
# drops Nan values
movies_df.dropna(inplace=True)

# defines Dtype of id
movies_df["id"] = movies_df['id'].astype(pd.Int64Dtype())

# renames id to movieId for later merge
movies_df = movies_df.rename(columns={'id' : 'movieId'})

movies_df

,movieId,title
0,862,Toy Story
1,8844,Jumanji
2,15602,Grumpier Old Men
3,31357,Waiting to Exhale
4,11862,Father of the Bride Part II
...,...,...
45461,439050,Subdue
45462,111109,Century of Birthing
45463,67758,Betrayal
45464,227506,Satan Triumphant


In [40]:
# merges the to dataframes
df = pd.merge(rating_df,movies_df,on='movieId')
df.head()

,userId,movieId,rating,title
0,1,1371,2.500,Rocky III
1,4,1371,4.000,Rocky III
2,7,1371,3.000,Rocky III
3,19,1371,4.000,Rocky III
4,21,1371,3.000,Rocky III


In [41]:
movie_ratingCount = (df.
     groupby(by = ['title'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'totalRatingCount'})
     [['title', 'totalRatingCount']]
    )
movie_ratingCount.head()

,title,totalRatingCount
0,!Women Art Revolution,2
1,'Gator Bait,1
2,'Twas the Night Before Christmas,2
3,...And God Created Woman,1
4,00 Schneider - Jagd auf Nihil Baxter,2


In [42]:
rating_with_totalRatingCount = df.merge(movie_ratingCount, left_on = 'title', right_on = 'title', how = 'left')
rating_with_totalRatingCount.head()

,userId,movieId,rating,title,totalRatingCount
0,1,1371,2.500,Rocky III,47
1,4,1371,4.000,Rocky III,47
2,7,1371,3.000,Rocky III,47
3,19,1371,4.000,Rocky III,47
4,21,1371,3.000,Rocky III,47


In [43]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(movie_ratingCount['totalRatingCount'].describe())

count   2794.000
mean      16.104
std       31.481
min        1.000
25%        1.000
50%        4.000
75%       15.750
max      324.000
Name: totalRatingCount, dtype: float64


In [44]:
popularity_threshold = 50
rating_popular_movie= rating_with_totalRatingCount.query('totalRatingCount >= @popularity_threshold')
rating_popular_movie.shape

(23969, 5)

In [50]:
rating_popular_movie.head()

,userId,movieId,rating,title,totalRatingCount
182,1,2294,2.000,Jay and Silent Bob Strike Back,53
183,5,2294,4.000,Jay and Silent Bob Strike Back,53
184,9,2294,2.000,Jay and Silent Bob Strike Back,53
185,15,2294,2.000,Jay and Silent Bob Strike Back,53
186,48,2294,3.000,Jay and Silent Bob Strike Back,53


In [45]:
movie_features_df=rating_popular_movie.pivot_table(index='title',columns='userId',values='rating').fillna(0)


In [ ]:
import 

In [46]:
movie_features_df.shape

(245, 665)

In [47]:
movie_features_df.head()

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
title,,,,,,,,,,,,,,,,,,,,,
"20,000 Leagues Under the Sea",0.000,0.000,0.000,3.000,0.000,2.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2001: A Space Odyssey,0.000,3.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,3.000,0.000,0.000,0.000,0.000,0.000,0.000
48 Hrs.,0.000,5.000,0.000,0.000,4.000,0.000,0.000,0.000,0.000,0.000,...,4.000,0.000,0.000,0.000,4.000,4.000,0.000,0.000,3.000,0.000
5 Card Stud,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.500,0.000,0.000,...,0.000,0.000,4.000,5.000,0.000,0.000,0.000,0.000,0.000,5.000
A Brief History of Time,0.000,0.000,0.000,0.000,0.000,2.000,0.000,0.500,4.000,5.000,...,0.000,0.000,0.000,0.000,0.000,0.000,4.000,0.000,0.000,0.000


In [51]:
from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(movie_features_df)

NearestNeighbors(algorithm='brute', metric='cosine')

In [53]:
A = model_knn.kneighbors_graph(movie_features_df)

In [54]:
A.toarray()

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])